# Frequency Containment Reserve and system efficiency
Our power system is extremely complex. How are thousands of power plants able to provide electricity to millions of consumers, specially since production and demand have to exactly match intermittently?

$$ P_{generation}(t) = P_{consumption}(t), \quad \forall~t \in T$$

The system operators take great care in the planing-phase to deal with this issue. Based on the forecasting of the power demand and generation from renewable sources, the dispatch of power plants is scheduled.
Nonetheless, there remains a mismatch caused by forecast errors or unforseable events, such as the unexpected shut-down of a big power plant. In that case special generators are assigned with power capacity reserved to compensate this difference.

These *preuqualified* generators balance the system by reacting to the system-frequency. This an equivalent measure, since power inbalance causes a deviation from the nominal frequency. [The basic logic behind](https://en.wikipedia.org/wiki/Swing_equation) is: if more power is drawn than the generators currently produce, the deficit will be covered form the inertia of the synchronous generators, slowing the rotors down. The other way around applies equally: if more power injected than the demand requires, the generator rotors will start to accelerate. By correcting the frequency and mantaning it close to the nominal frequency the stability of the grid is assured. Big frequency deviation could cause system cut-offs and blackouts.

The frequency control is dealt with mechanisms that are enforced in different stages. 
The [Frequency Containtment Reserve (FCR)](https://www.next-kraftwerke.com/knowledge/frequency-containment-reserve-fcr) is the primary reserve and reacts to disturbances instantly, acting as a safety net of the power system.
If the deviation persists for a few minutes, the Frequency Restoration Reserve (FRR) steps in to alleviate the FCR. The FFR itself is comprised from the secondary reserve followed by the tertiary reserve.

Battery energy storage systems provide flexible control and fast reaction times, so they are very well suited for providing FCR services. FCR is the most relevant application for large grid-scale BESS, they have now been widely adopted and account over x MW of BESS providing the service in Germany.


<!-- One of the [first proposed](https://doi.org/10.1109/TEC.1986.4765732) applications for BESS providing ancillary services. -->

In this notebook we will:
* Analyze a frequency profile
* The power response of FCR prequalified generators
* Explore how battery energy storage
* Evaluate effect of the power electronics efficiency in the system performance

In [ ]:
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
pd.options.display.float_format = "{:.6f}".format
pd.options.plotting.backend = "plotly"
template = "plotly_white"
# template = "plotly_dark"

## Frequency profile

For our first step into exploring FCR, we will take a look into the frequency. We will use a [dataset](https://www.50hertz.com/de/Transparenz/Kennzahlen/Regelenergie/ArchivNetzfrequenz) provided by the TSO *50hertz*, which shows mains frequency measurements in 1 second resolution. 

As an example, let's take a look into the data for January 2019.

In [ ]:
df = pd.read_csv("../data/201901_Frequenz.csv", usecols=[0,1,3], parse_dates=[[0,1]], index_col=0, names=["date", "time", "f"])

In [ ]:
df

In [ ]:
df.resample("1Min").mean().plot(template=template, height=50) # downsampling for faster/lighter plotting

<!-- A small description, rules on permitted frequency deviation. -->

The [ENTSO-E network code](https://www.entsoe.eu/network_codes/) defines a standard frequency range of $\pm$ 0.05 Hz from the nominal frequency (the range within the grid should be operated) and specifies that the frequency should remain within this range for ~97% of the time in a year.

Let's take a deeper look into the profile with some stats.

<div class="alert alert-block alert-info">
<b>Task!</b> Examine the frequency profile:
<ul>
    <li> Plot a histogram of the frequency distribution. </li>
    <li> Find the min and max frequencies, the mean and standard deviation. </li>
    <li> Find for how much of the time the frequency is within the standard frequency range. </li>
    <li> Comment your observations in a markdown cell. </li>
</ul>
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> Check the documentation for pandas-plotting with plotly.
</div>

In [ ]:
# task

In [ ]:
# task

In [ ]:
# task

## FCR power response

The FCR is performed through a frequency droop control, this means a prequalified generator reacts with power proportionally to the frequency deviation. 
The details on the droop control implementation varies from country to country.
In Germany the droop control for FCR is regulated as following: the system reacts with power proportional to the frequency deviation up to a maximum of $\pm 200~\text{mHz}$ where the system has to provide full power. For deviations below $\pm 10~\text{mHz}$ a deadband is defined, in which the system does not have to react.

$$
P_g =
\begin{cases}
    0.0~\text{MW} & \quad \text{if }|\Delta f| < 10~\text{mHz} \\
    \pm P_n & \quad \text{if }|\Delta f| > 200~\text{mHz} \\
    P_n \frac{\Delta f}{f_n  S} & \quad \text{else}
\end{cases}
% \qquad \qquad \text{NSE (1.1)}
$$

The droop $S$ describes the slope that determines the amount of power required to counterbalance the frequency deviation.

$$ S = \frac{-\Delta f / f_n}{\Delta P / P_n} $$


Following, we will take as example a *prequalified generator* with $P_n = 1~\text{MW}$. Let's take a look on the droop control characteritics of the system.

<div class="alert alert-block alert-info">
<b>Task!</b> Explore the power response:
<ul>
    <li> Calculate the droop <code>S</code> based on the prequalified power (1 MW) and the system requirements.
    <li> Implement a function <code>fcr_power</code> that takes a frequency <code>f</code> and returns a power <code>Pg</code> based on the nominal frequency <code>fn</code>, droop <code>S</code> and frequency dead-band <code>fdb</code> </li>
    <li> Plot the droop response curve (x-axis frequency, y-axis P/Pn) </li>
</ul>
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b>
<ul>
    <li> The resulting droop S is 0.4 % </li>
    <li> Plot the droop response curve with frequency deviations Δf from -220 mHz to 220 mHz.</li>
</ul>
</div>

In [ ]:
# parameters
fn = 50.0     # nominal frequency [Hz]
fdb = 10.0e-3 # frequency dead-band [Hz]
delta_f_max = 200.0e-3 # maximum frequency deviation [Hz]

Pn = 1.0   # Nominal power [MW]

In [ ]:
# task
S = ...

In [ ]:
# task
def fcr_power(f, fn, Pn, S, fdb):
    ...

In [ ]:
# task

Let's add the FCR power response to our DataFrame and plot it together with the frequency.

In [ ]:
df["power"] = [fcr_power(f, fn, Pn, S, fdb) for f in df["f"]]

In [ ]:
def plot_fcr(df, inverse_power=False, show_deadband=True, **kwargs):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(**kwargs)

    fig.add_trace(go.Scatter(x=df.index, y=df["f"], name="f"))
    fig.update_yaxes(title="Frequency [Hz]")
    
    fig.add_trace(go.Scatter(x=df.index, y=df["power"], name="power"), secondary_y=True)
    fig.update_yaxes(title="Power [MW]", secondary_y=True)

    if show_deadband:
        fdb = 10.0e-3
        fig.add_hline(y = 50 - fdb, opacity=0.5, line_dash="dot")
        fig.add_hline(y = 50 + fdb, opacity=0.5, line_dash="dot")
    
    # Align y-axes: 50 Hz and 0 MW
    fig.update_yaxes(range=[50-0.2, 50+0.2])
    
    p_max = 1
    fig.update_yaxes(range=[-p_max, p_max], secondary_y=True)
    
    if inverse_power:
        fig.update_yaxes(range=[p_max, -p_max], secondary_y=True)
    else:
        fig.update_yaxes(range=[-p_max, p_max], secondary_y=True)

    return fig

In [ ]:
# plotting 1 day
plot_fcr(df.loc["2019-01-01 12"], inverse_power=False, template=template, height=500)

## Battery Energy Storage for FCR

Following, we will consider a battery energy storage system to provide FCR. The storage is fully capable to react to the frequency disturbances, but its operation is constrained with the available energy capacity. We can visualize the progression of the energy flow involved and portray a qualitative trend of the storage SOC.

In [ ]:
# delivered energy
energy = df["power"].cumsum() / 3600 # MW -> MWh
energy.resample("1Min").mean().plot(template=template, labels={"value": "Energy [MWh]"})

In [ ]:
print(f"Pseudo capacity: {energy.max() - energy.min():.2f} MWh")

In [ ]:
print(f"Final energy offset: {energy[-1]:.2f} MWh")

This cannot be directly translated to what capacity is really required, since it only portrays a single (not necessarily representative) profile. It is rather an approximate representation of the potential requirements. Furthermore, the here portrayed system is ideal, but in reality systems losses also play an important role in the overall performance.

## System efficiency

For BESS, the conversion losses from the inverters take the most significant share of the total losses, therefore we will incorporate them into the analysis. Important to note is that the efficiency of the inverters is not constant, but it also depends on the input AC power.
This *efficiency curve* can be modelled with the following equation by [Notton et al.](https://www.sciencedirect.com/science/article/abs/pii/S0960148109003085?via%3Dihub)

$$ \eta (P) = \frac{P/P_n}{P/P_n + k_0 + k_2 * (P/P_n)^2} $$

Where $P/P_n$ is the ratio of the input AC power to the nominal value and $k_0$ and $k_2$ are fitting parameters that describe the characteristics of the inverter efficiency. For our inverter model we assume the following: $k_0 = 0.02$ and $k_2 = 0.005$.

Our large scale storage system itself is conformed from multiple AC-strings connected in parallel, **8 units** in total each with an individual inverter. As a starting point we will operate them with a very simple strategy: equally distribute the power among all units.


<div class="alert alert-block alert-info">
<b>Task!</b> :
<ul>
    <li> Implement the function <code>unit_efficiency</code> that takes the absolute ratio |P/Pn| (<code>power_ratio</code>) and returns the inverter efficiency. </li>
    <li> Implement a function <code>power_converter_uniform</code> that takes the current AC power setpoint <code>power</code>, the total installed nominal power <code>power_max</code> and the number of units <code>units</code>, distributes the AC power to the single units, and returns the aggregated DC power. </li>
</ul>
</div>

In [ ]:
# task
def unit_efficiency(power_ratio, params=(0.02, 0.005)):
    k0, k2 = params
    ...

In [ ]:
# task
def power_converter_uniform(power, power_max, units):
    ...

Let's plot the efficiency curve...

In [ ]:
def plot_efficiency_curve(*converters, power_max=1, units=8, **kwargs):
    fig = go.Figure()
    fig.update_layout(xaxis_title="P/P_n", yaxis_title="Efficiency", **kwargs)

    power_range = np.linspace(0, power_max, num=1000)
    power_normalized = power_range / power_max
    
    for converter in converters:
        power_dc = np.array([converter(power, power_max, units) for power in power_range])
        power_dc[0] = 1 # avoid division by zero
        eff = power_range / power_dc
        fig.add_trace(go.Scatter(x=power_normalized, y=eff, name=converter.__name__))

    return fig

In [ ]:
plot_efficiency_curve(power_converter_uniform, units=8, template=template, height=500)

... and compute the FCR power response from the DC side, which directly translates to the power requirements from the battery modules.

In [ ]:
df["power_dc_uniform"] = [power_converter_uniform(ac_power, 1.0, 8) for ac_power in df["power"]]

In [ ]:
df.loc["2019-01-01 01", ["power", "power_dc_uniform"]].plot(template=template, labels={"value": "Power [MW]"}, height=500)

In [ ]:
energy = df[["power", "power_dc_uniform"]].cumsum() / 3600
energy.resample("1Min").mean().plot(template=template, labels={"value": "Energy [MWh]"})

We see from the DC side a significant divergence, which will affect the system performance. The system would not be able to provide the same amount of FCR service, or would have to be oversize in order to do so. The losses accumulate and we also see the *delivered energy* drifting upwards with the time. This deficit will have to be covered by buying electricity in the intra-day energy market (IDM), to bring the storage back to operating conditions. These additional operating costs will also decrease the overal profitability.

In [ ]:
# Pseudo capacity [MWh]
energy.max() - energy.min()

In [ ]:
# Final energy offset [MWh]
energy.iloc[-1]

## Improving the system efficiency

Part of the problem comes from the overall *low power* requirements of the FCR operation and the bad efficiency of the inverters at a partial load. We will try to improve the performance through a *cascading* strategy: use the minimum number of units required to cover the power setpoint and then distribute the load between these units equally.

<div class="alert alert-block alert-info">
<b>Task!</b> 
<ul>
    <li> Implement the cascade power distribution strategy. Write a function <code>power_converter_cascade</code> that takes the AC power setpoit <code>power</code>, the total installed nominal power <code>power_max</code> and the number of units <code>units</code>, distributes the AC power to the single units, and returns the aggregated DC power. </li>
    <li> Implement a function <code>roundtrip_efficiency</code> to computes therountrip efficiency based on AC and DC input profiles. </li>
    <li> Compare both strategies. Explain in a markdwon cell, how the cascade strategy can improve the system efficiency and the overall performance. </li>
</ul>
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> For the roundtrip efficiency calculate first the charge/discharge efficiencies separately. 
</div>

In [ ]:
def power_converter_cascade(power, power_max, units):
    ...

Let's plot both efficiency curves together...

In [ ]:
plot_efficiency_curve(power_converter_uniform, power_converter_cascade, units=8, template=template, height=500)

... and compute the DC power and energy balance from the cascading strategy.

In [ ]:
df["power_dc_cascade"] = [power_converter_cascade(ac_power, 1.0, 8) for ac_power in df["power"]]

In [ ]:
energy = df[["power", "power_dc_uniform", "power_dc_cascade"]].cumsum() / 3600
energy.resample("1Min").mean().plot(template=template, labels={"value": "Energy [MWh]"})

In [ ]:
# pseudo required capacity
energy.max() - energy.min()

In [ ]:
# Final energy offset
energy.iloc[-1]

Finally, let's compare both strategies:

In [ ]:
#task
def roundtrip_efficiency(power_ac, power_dc):
    ...

In [ ]:
# task